### 1. 概述
#### 1.1 Numpy和Pandas的区别
||Numpy|Pandas|
|:----|:----|:----|
|作用|数值计算|数据处理|
|主要数据结构|ndarray|Series,DataFrame|
|数据类型|ndarry同构|Series同构|


### 2. 基本数据结构
#### 2.1 Series
+ 带标签(index)的一维数组
+ 创建方式：pd.Series(data, index=index) (index形式灵活，基本都可以识别出来)
    + data不可为set,但index可为set,此时index无序，随机
+ data支持的数据类型:
    + Python 字典
    + 多维数组(不能是set，set无序)
        + 未设置index时：
           + Python 版本 >= 3.6 且 Pandas 版本 >= 0.23 Series按字典的插入顺序排序索引
           + Python < 3.6 或 Pandas < 0.23 Series按字母顺序排序字典的键（key）列表
        + 设置index时：
           + 按索引标签提取data里对应的值
    + 标量值（如1）
        + data为标量值时，必须提供index，Series按index长度重复
+ 基本操作：
    + 与ndarray类似，支持大多数Numpy函数，同时支持索引切片

In [17]:
import pandas as pd
import numpy as np
pd.Series(np.array(range(5)), (1,2,3,4,5)) # index 形式灵活，基本都可以识别出来

### data 为多维数组
pd.Series((1,2,3))

### data 为字典
pd.Series({'b': 1, 'a': 0, 'c': 2})
pd.Series({'b': 1, 'a': 0, 'c': 2}, index=['1', 'b', 'c', 'd']) #按索引标签提取

### data为标量值
pd.Series(1, index=['a', 'b'])  #必须提供index，按index长度重复该标量值

a    1
b    1
dtype: int64

In [33]:
### 与numpy操作类似
a = pd.Series(np.array([3,2,1,0]))
a.dtype
a[0]    #第一个值
a[[0]]  #index为0的值


3